<a href="https://colab.research.google.com/github/c-biancone/ISCH-370_Project/blob/main/ISCH370_Group2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! git remote add origin https://github.com/c-biancone/ISCH-370_Project.git

fatal: not a git repository (or any of the parent directories): .git


Import datasets

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import DoT county transportation profiles
import pandas as pd
countyTrans = pd.read_csv('/content/drive/MyDrive/Semester_6/Data_Science/data_sets/County_Transportation_Profiles.csv')

# import DoT trips by distance
# not right now don't want to slow this down too much

In [3]:
countyTrans

,County FIPS,County Name,State FIPS,State Name,Primary and Commercial Airports,Non-Commercial -Civil Public Use Airports and Seaplane base,Non-Commercial -Other Aerodromes,Number of Bridges,% of Poor Condition Bridges,Number of business establishments,...,Number of workers from other counties who commute to work in the county,Number of resident workers who commute to work in other counties,Number of resident workers who commute within county,Number of resident workers,Number of residents,Total Docks,Total Marinas,Route miles of freight railroad,% of Medium to Fair Condition Bridges,Route miles of passenger railroad and rail transit
0,1001,Autauga County,1,Alabama,0.0,1.0,0.0,114.0,0.035088,869.0,...,15576,15417,8710,24300,55050,NaN,NaN,50.7,0.964912,0.0
1,1003,Baldwin County,1,Alabama,0.0,4.0,28.0,360.0,0.019444,5384.0,...,24011,17748,62235,86310,199510,1.0,0.0,30.4,0.980556,0.0
2,1005,Barbour County,1,Alabama,0.0,2.0,2.0,193.0,0.031088,455.0,...,2368,1871,6395,8785,26615,0.0,0.0,14.7,0.968912,0.0
3,1007,Bibb County,1,Alabama,0.0,1.0,0.0,126.0,0.000000,284.0,...,4868,4816,3410,8280,22570,NaN,NaN,25.1,1.000000,5.5
4,1009,Blount County,1,Alabama,0.0,1.0,1.0,183.0,0.087432,698.0,...,14171,13981,7205,21350,57705,NaN,NaN,13.1,0.912568,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,Sweetwater County,56,Wyoming,1.0,1.0,1.0,195.0,0.071795,1248.0,...,740,469,21855,22595,44810,NaN,NaN,220.7,0.928205,0.0
3138,56039,Teton County,56,Wyoming,1.0,0.0,7.0,117.0,0.136752,2140.0,...,775,360,13220,14010,22625,NaN,NaN,NaN,0.863248,NaN
3139,56041,Uinta County,56,Wyoming,0.0,2.0,1.0,132.0,0.181818,560.0,...,1632,1093,7685,9310,20895,NaN,NaN,74.2,0.818182,0.0
3140,56043,Washakie County,56,Wyoming,0.0,1.0,3.0,61.0,0.114754,346.0,...,339,300,3300,3640,8350,NaN,NaN,28.0,0.885246,0.0


.  ..  .config	data_sets  drive  .ipynb_checkpoints  sample_data
